In [9]:
## data format
import os
import pandas as pd
import numpy as np
import polars as pl
import pickle

## preprocessing features
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import string
from nltk.corpus import stopwords

## predictor
from catboost import CatBoostRegressor

## evaluation server
import kaggle_evaluation.mcts_inference_server


targets = ['num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1', 'utility_agent1']
target = 'utility_agent1'
nl_cols = ["EnglishRules", "LudRules"]

with open("/root/kag_comp/UM_MCTS/trained_model.pkl", 'rb') as f :
    predictr = pickle.load(f)

with open("/root/kag_comp/UM_MCTS/transformr.pkl", 'rb') as f :
    pipes = pickle.load(f)

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    NL_test = test[nl_cols]
    XX_NL_list = []

    for i, col in enumerate(nl_cols) :
        XX_NL_col = pipes[i].transform(NL_test[col].to_pandas())
        XX_NL_list.append(XX_NL_col.toarray())

    XX_NL = pd.DataFrame(np.hstack(XX_NL_list))
    XX = pd.concat([test.drop(nl_cols).to_pandas(), XX_NL], axis = 1)

    predictions = pd.Series(predictr.predict(XX)).map(lambda x : x if (x <= 1) & (x >= -1) else 1 if x > 1 else -1)

    return sample_sub.with_columns(pl.Series('utility_agent1', predictions))

# 인퍼런스 서버 설정
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/root/kag_comp/UM_MCTS/Data/test.csv',
            '/root/kag_comp/UM_MCTS/Data/sample_submission.csv'
        )
    )